In [1]:
import sys
sys.path.append("../codesearchnet")


import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [102]:
import os.path as osp


# import torch
# import torch.nn.functional as F
# from torch_geometric.datasets import Planetoid
# import torch_geometric.transforms as T
# from torch_geometric.nn import GCNConv, GAE, VGAE
# from torch_geometric.utils import train_test_split_edges


import swifter
import fasttext as ft
import numpy as np
import networkx as nx
from livelossplot import PlotLosses
from livelossplot.outputs import TensorboardTFLogger
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


import tf_geometric as tfg

from graph_nets import blocks
from graph_nets import graphs
from graph_nets import modules
from graph_nets import utils_np
from graph_nets import utils_tf
import sonnet as snt

import dgl

from code_parser import *
from data_reader import get_data_df
from siamese_model_keras import *
# from keras_preprocessing_helper import *

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [103]:
parser = get_parser(lang="python", so_path="../resources/csnet_parse_build.so")

In [104]:
df = get_data_df("../resources/data/", ["python"] , ["test"])

In [112]:
g = parse_program(df['code'][0], parser=parser)

In [113]:
g.number_of_edges(), g.number_of_nodes()

(95, 96)

In [114]:
for node in g.nodes():
    g.add_node(node, data=code_ft[node.name])
g = nx.convert_node_labels_to_integers(g)
g_dgl = dgl.DGLGraph()
g_dgl.from_networkx(g, node_attrs=['data'])

In [94]:
g_spicy = nx.to_scipy_sparse_matrix(g, format='coo')
data_dict_0 = {
    "nodes": [code_ft[x.name] for x in g.nodes()],
    "senders": g_spicy.col,
    "receivers": g_spicy.row,
}
g = utils_tf.data_dicts_to_graphs_tuple([data_dict_0])

In [95]:
import sonnet as snt
from graph_nets import modules
from graph_nets import utils_tf

NUM_LAYERS = 2  # Hard-code number of layers in the edge/node/global models.
LATENT_SIZE = 16  # Hard-code latent layer sizes for demos.


def make_mlp_model():
    """Instantiates a new MLP, followed by LayerNorm.
    The parameters of each new MLP are not shared with others generated by
    this function.
    Returns:
      A Sonnet module which contains the MLP and LayerNorm.
    """
    return snt.Sequential([
        snt.nets.MLP([LATENT_SIZE] * NUM_LAYERS, activate_final=True),
        snt.LayerNorm(axis=-1, create_offset=True, create_scale=True)
    ])


class MLPGraphIndependent(snt.Module):
    """GraphIndependent with MLP edge, node, and global models."""

    def __init__(self, name="MLPGraphIndependent"):
        super(MLPGraphIndependent, self).__init__(name=name)
        self._network = modules.GraphIndependent(
            edge_model_fn=make_mlp_model,
            node_model_fn=make_mlp_model,
            global_model_fn=make_mlp_model)

    def __call__(self, inputs):
        return self._network(inputs)

In [96]:
model = MLPGraphIndependent()

In [97]:
g = utils_tf.set_zero_global_features(g, 1)
g = utils_tf.set_zero_edge_features(g, 1)

In [99]:
out = model(g)

In [100]:
out

GraphsTuple(nodes=<tf.Tensor: shape=(96, 16), dtype=float32, numpy=
array([[ 0.25297284,  2.3808131 ,  0.00869268, ..., -0.6934152 ,
         1.6180739 , -0.6934152 ],
       [ 1.1218919 ,  0.5875819 , -0.90184057, ...,  0.28387415,
        -0.90184057, -0.90184057],
       [-0.21839023,  0.1266272 , -0.6516292 , ..., -0.19566432,
        -0.6516292 , -0.6516292 ],
       ...,
       [-0.6414861 , -0.6414861 ,  1.1386132 , ...,  2.573797  ,
        -0.6414861 , -0.6414861 ],
       [-0.6414861 , -0.6414861 ,  1.1386132 , ...,  2.573797  ,
        -0.6414861 , -0.6414861 ],
       [-0.6414861 , -0.6414861 ,  1.1386132 , ...,  2.573797  ,
        -0.6414861 , -0.6414861 ]], dtype=float32)>, edges=<tf.Tensor: shape=(95, 16), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype

In [7]:
g_spicy = nx.to_scipy_sparse_matrix(g, format='coo')

In [16]:
tfg_graph = tfg.Graph(x=[code_ft[x.name] for x in g.nodes()], edge_index=[graph_spicy.row, graph_spicy.col])

In [31]:
gin0 = tfg.layers.GIN(128, activation=tf.nn.relu)
gin1 = tfg.layers.GIN(128, activation=tf.nn.relu)
mlp = tf.keras.Sequential([
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512)
])

In [37]:
h = gin0([tfg_graph.x, tfg_graph.edge_index])
print(h.shape)
h = gin1([h, tfg_graph.edge_index])
print(h.shape)
h = mlp(h)
print(h.shape)

(96, 128)
(96, 128)
(96, 512)


In [26]:
h =  dropout(h)

In [28]:
h = gcn1([h, tfg_graph.edge_index])

In [29]:
h

<tf.Tensor: shape=(96, 10), dtype=float32, numpy=
array([[-6.00954711e-01, -1.00283003e+00,  2.16055110e-01,
         4.43325549e-01, -2.52041608e-01,  6.51502430e-01,
         7.83913374e-01,  3.93085778e-01,  9.88498092e-01,
         7.55274177e-01],
       [-7.01416671e-01, -1.67606640e+00, -1.05941623e-01,
         9.09001827e-01,  5.72662473e-01,  5.78170896e-01,
         1.85517633e+00,  9.68147397e-01,  1.58618426e+00,
         2.80255616e-01],
       [-4.16202515e-01, -2.70323157e-01, -2.09212109e-01,
         2.72532314e-01,  2.94887483e-01,  4.41922903e-01,
         6.86850667e-01,  2.37496153e-01,  5.68876863e-02,
        -9.90092009e-02],
       [-1.82056770e-01, -1.19422090e+00, -4.92238581e-01,
         1.59415352e+00,  3.44475478e-01,  4.58703607e-01,
         1.62732816e+00,  7.51324117e-01,  1.51683497e+00,
         1.56842649e-01],
       [-1.37031698e+00, -1.31611776e+00, -8.98292005e-01,
         1.38866615e+00,  1.25485790e+00,  9.16698039e-01,
         2.83705354e

In [ ]:
def get_data_from_graph(g: nx.DiGraph, y=None) -> Data:
    
    graph_spicy = nx.to_scipy_sparse_matrix(g, format='coo')
   
    tf_g = tfg.Graph(
        x=[code_ft[x.name] for x in g.nodes()],
        edge_index=[graph_spicy.row, graph_spicy.col]
        y=None
    )
    
    return tf_g



In [ ]:
embeddings_dim_c, embeddings_dim_q = 256, 256
max_len_code, max_len_query = 48, 28
batch_size = 32
epochs = 10

exp_name = "try"

In [10]:
# load emebeddigs for for query and code
query_ft = ft.load_model("../resources/python_processed/query_ft.bin")
code_ft = ft.load_model("../resources/python_processed/code_no_ast.bin")

In [ ]:
train_gen = get_generator("../resources/data/", ["python"] , ["train"], max_len_query, max_len_code, query_ft, code_ft)
valid_gen = get_generator("../resources/data/", ["python"] , ["valid"], max_len_query, max_len_code, query_ft, code_ft)
test_gen = get_generator("../resources/data/", ["python"] , ["test"], max_len_query, max_len_code, query_ft, code_ft)

In [ ]:
train_ds = tf.data.Dataset.from_generator(train_gen,  
                                    (tf.float32, tf.float32),  
                                    (tf.TensorShape([max_len_query, embeddings_dim_q]),
                                     tf.TensorShape([max_len_code, embeddings_dim_c])))

valid_ds = tf.data.Dataset.from_generator(valid_gen,  
                                    (tf.float32, tf.float32),  
                                    (tf.TensorShape([max_len_query, embeddings_dim_q]),
                                     tf.TensorShape([max_len_code, embeddings_dim_c])))

test_ds = tf.data.Dataset.from_generator(test_gen,  
                                    (tf.float32, tf.float32),  
                                    (tf.TensorShape([max_len_query, embeddings_dim_q]),
                                     tf.TensorShape([max_len_code, embeddings_dim_c])))

In [ ]:
train_ds = train_ds.batch(batch_size).prefetch(batch_size*2)
valid_ds = valid_ds.batch(batch_size).prefetch(batch_size*2)
test_ds = test_ds.batch(batch_size).prefetch(batch_size*2)

In [ ]:
model = get_model_lstm(max_len_query, max_len_code, embeddings_dim_q, embeddings_dim_c)

In [ ]:
optimizer = tf.optimizers.Adam()

In [ ]:
liveloss = PlotLosses(outputs=[TensorboardTFLogger("./exp/tb/", run_id=exp_name)])
logs = {}
best_val_loss = 100000.0

for epoch in range(epochs):

    # TRAINING
    losses = []
    mrrs = []
    for x in train_ds:
        
        with tf.GradientTape() as tape:
            logits = model(x)
            loss_value = softmax_loss(None, logits)

        mrr_value = mrr(None, logits)

        # calculate gradient
        gradients = tape.gradient(loss_value, model.trainable_variables)

        # Update the weights
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        losses.append(loss_value)
        mrrs.append(mrr_value)

        print(f"Epoch: {epoch}; Loss: {loss_value}; MRR: {mrr_value} <- Train", end="\r")
        
    logs['loss'] = np.mean(losses)
    logs['mrr'] = np.mean(mrrs)
    
    
    # VALIDATION
    losses = []
    mrrs = []
    for x in valid_ds:
        
        logits = model(x)
        
        loss_value = softmax_loss(None, logits)
        mrr_value = mrr(None, logits)
        
        losses.append(loss_value)
        mrrs.append(mrr_value)
        
        print(f"Epoch: {epoch}; Loss: {loss_value}; MRR: {mrr_value} <- Test", end="\r")
        
    logs['val_loss'] = np.mean(losses)
    logs['val_mrr'] = np.mean(mrrs)
    
    if logs['val_loss'] < best_val_loss:
        best_val_loss = logs['val_/loss']
        model.save(f"exp/{exp_name}.h5")
    
    liveloss.update(logs)
    liveloss.send()

In [ ]:
model = tf.keras.models.load_model(f"exp/{exp_name}.h5")

In [ ]:
    losses = []
    mrrs = []
    for x in get_dataset("test"):
        
        logits = model(x)
        
        loss_value = loss_(None, logits)
        mrr_value = mrr(None, logits)
        
        losses.append(loss_value)
        mrrs.append(mrr_value)
        
        print(f"Epoch: test; Loss: {loss_value}; MRR: {mrr_value} <- Test", end="\r")

In [ ]:
np.mean(mrrs)

## Predit/Demo

In [ ]:
from annoy import AnnoyIndex
import random


In [ ]:
model = tf.keras.models.load_model(f"exp/{exp_name}.h5")

In [ ]:
model.summary()

In [ ]:
query_encoder = tf.keras.models.Model(model.get_layer('input_1').input, model.get_layer('lstm').output)
code_encoder = tf.keras.models.Model(model.get_layer('input_2').input, model.get_layer('lstm_1').output)

In [ ]:
# get feature vector for both query and code
querys = []
codes = []

for q,c in test_ds.take(1000):
    querys.extend(query_encoder(q))
    codes.extend(code_encoder(c))

In [ ]:
# create an index for fast matching of vectors

t = AnnoyIndex(256, 'angular')
for i in range(len(codes)):
    t.add_item(i, codes[i])

In [ ]:
# build the tree

t.on_disk_build("exp/code_no_ast_embedding_screath_try.annoy_on_disk")
t.build(100)

In [ ]:
# save the tree for later reference
t.save('exp/code_no_ast_embedding_screath_try.annoyme')

In [ ]:
result = t.get_nns_by_vector(codes[2155], n=2, include_distances=False)
result

In [ ]:
correct = 0 
for i in range(len(querys)):
    result = t.get_nns_by_vector(querys[i], n=10, include_distances=False)
    if i in result:
        correct += 1

In [ ]:
correct/len(querys)

In [ ]:
## snippet for loading a tree
from annoy import AnnoyIndex
t = AnnoyIndex(256, 'angular')
t.load('exp/code_no_ast_embedding_screath_try.annoyme')

In [ ]:
with open("../resources/data/python_dedupe_definitions_v2.pkl", "rb") as f:
    import pickle 
    definations = pickle.load(f)